# 📈 DEEPCHECKS CRIME LA - NIVEAU 2 : DRIFT ET DISTRIBUTION

## 🎯 Objectif
Analyser les distributions et détecter le drift entre train/test pour les données de criminalité :
- Vérification de la stabilité des features entre train/test
- Détection de drift dans les labels (types de crimes)
- Analyse des propriétés statistiques des données
- Validation de la représentativité du split

---

## 📦 Imports et Configuration

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Ajouter src au path
sys.path.append(os.path.join(os.getcwd(), '..'))

print("✅ Imports de base effectués")

In [ ]:
# Deepchecks Tabular pour drift
try:
    from deepchecks.tabular import Dataset
    from deepchecks.tabular.suites import train_test_validation
    from deepchecks.tabular.checks import (
        TrainTestFeatureDrift, TrainTestLabelDrift,
        WholeDatasetDrift, FeatureLabelCorrelation,
        TrainTestSamplesMix, NewLabelTrainTest,
        CategoryMismatchTrainTest, DateTrainTestLeakageDuplicates
    )
    DEEPCHECKS_AVAILABLE = True
    print("✅ Deepchecks importé")
except ImportError as e:
    print(f"⚠️ Deepchecks non disponible: {e}")
    DEEPCHECKS_AVAILABLE = False

# ML pour le split
from sklearn.model_selection import train_test_split

In [ ]:
# Configuration des chemins
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = BASE_DIR / 'data'
REPORTS_DIR = BASE_DIR / 'reports' / 'deepchecks'
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print("="*80)
print("📈 DEEPCHECKS CRIME LA - NIVEAU 2 : DRIFT ET DISTRIBUTION")
print("="*80)
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📁 Base: {BASE_DIR}")
print()

---
## 📥 Chargement et Préparation des Données

In [ ]:
def load_and_prepare_crime_data():
    """Charge et prépare les données avec preprocessing si nécessaire"""
    print("📦 Chargement et préparation des données")
    print("-" * 80)
    
    # Chercher données processed en priorité
    processed_files = list((DATA_DIR / 'processed').glob('*.csv'))
    raw_files = list((DATA_DIR / 'raw').glob('*.csv'))
    
    if processed_files:
        data_file = processed_files[0]
        df = pd.read_csv(data_file)
        print(f"✅ Données processed chargées: {data_file.name}")
    elif raw_files:
        data_file = raw_files[0]
        df = pd.read_csv(data_file)
        print(f"✅ Données raw chargées: {data_file.name}")
        
        # Appliquer preprocessing basique si nécessaire
        try:
            from src.data.preprocessing import clean_data
            df = clean_data(df)
            print("✅ Preprocessing appliqué")
        except Exception as e:
            print(f"⚠️ Preprocessing automatique échoué: {e}")
    else:
        raise FileNotFoundError("Aucun fichier CSV trouvé dans data/")
    
    print(f"   Shape finale: {df.shape}")
    print(f"   Colonnes: {df.columns.tolist()[:10]}{'...' if len(df.columns) > 10 else ''}")
    
    return df

In [ ]:
# Charger les données
df_crime = load_and_prepare_crime_data()

---
## 🔄 Création du Split Train/Test pour Drift Analysis

In [ ]:
def create_train_test_split_for_drift(df):
    """Crée un split train/test représentatif pour analyse de drift"""
    print("📝 Création du split Train/Test pour drift analysis")
    print("-" * 80)
    
    # Identifier label et features
    label_col = None
    if 'Crime_Group' in df.columns:
        label_col = 'Crime_Group'
    elif 'Crm Cd Desc' in df.columns:
        # Mapper vers groupes si pas encore fait
        df = df.copy()
        try:
            from src.models.train import map_crime_group_4
            df['Crime_Group'] = df['Crm Cd Desc'].apply(map_crime_group_4)
            label_col = 'Crime_Group'
        except:
            label_col = 'Crm Cd Desc'
    
    # Features pour ML
    feature_cols = []
    potential_features = ['Hour', 'Day_of_week', 'Month_num', 'LAT', 'LON', 'Vict Age', 'AREA']
    
    for col in potential_features:
        if col in df.columns:
            feature_cols.append(col)
    
    if not feature_cols:
        raise ValueError("Aucune feature ML trouvée. Vérifiez le preprocessing.")
    
    # Préparer les données pour split
    df_ml = df[feature_cols + [label_col]].dropna()
    
    X = df_ml[feature_cols]
    y = df_ml[label_col]
    
    # Split stratifié
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"✅ Split effectué:")
    print(f"   Train: {X_train.shape[0]} échantillons")
    print(f"   Test:  {X_test.shape[0]} échantillons")
    print(f"   Features: {feature_cols}")
    print(f"   Label: {label_col}")
    
    # Vérification distribution
    train_dist = y_train.value_counts(normalize=True).sort_index()
    test_dist = y_test.value_counts(normalize=True).sort_index()
    
    print(f"\n📊 Distribution des labels:")
    for label in train_dist.index:
        print(f"   {label}: Train {train_dist[label]:.3f} | Test {test_dist[label]:.3f}")
    
    return X_train, X_test, y_train, y_test, feature_cols, label_col

In [ ]:
# Créer le split
X_train, X_test, y_train, y_test, features, label_column = create_train_test_split_for_drift(df_crime)

---
## 📊 Création des Datasets Deepchecks

In [ ]:
def create_deepchecks_datasets(X_train, X_test, y_train, y_test, features):
    """Crée les datasets Deepchecks pour train et test"""
    print("📝 Création des datasets Deepchecks")
    print("-" * 80)
    
    # Reconstituer DataFrames avec labels
    train_df = X_train.copy()
    train_df['__label__'] = y_train.values if hasattr(y_train, 'values') else list(y_train)
    
    test_df = X_test.copy()
    test_df['__label__'] = y_test.values if hasattr(y_test, 'values') else list(y_test)
    
    # Identifier features catégorielles
    cat_features = []
    for col in features:
        if train_df[col].dtype == 'object' or col in ['Day_of_week', 'Month_num', 'AREA']:
            cat_features.append(col)
    
    print(f"✅ Datasets créés:")
    print(f"   Features numériques: {[f for f in features if f not in cat_features]}")
    print(f"   Features catégorielles: {cat_features}")
    
    if DEEPCHECKS_AVAILABLE:
        train_dataset = Dataset(
            train_df, 
            label='__label__', 
            cat_features=cat_features,
            index_name=None
        )
        test_dataset = Dataset(
            test_df, 
            label='__label__', 
            cat_features=cat_features,
            index_name=None
        )
        return train_dataset, test_dataset
    else:
        return train_df, test_df

In [ ]:
# Créer les datasets
train_dataset, test_dataset = create_deepchecks_datasets(X_train, X_test, y_train, y_test, features)

---
## 📈 NIVEAU 2 : ANALYSE DRIFT ET DISTRIBUTION

### Suite de validation train/test adaptée au crime

Les checks exécutés :
1. **TrainTestFeatureDrift** : Détecte le drift dans les features entre train et test
2. **TrainTestLabelDrift** : Vérifie le drift dans la distribution des labels
3. **WholeDatasetDrift** : Analyse globale du drift du dataset
4. **FeatureLabelCorrelation** : Corrélation features/labels cohérente
5. **TrainTestSamplesMix** : Détecte un mélange problématique train/test

In [ ]:
def run_drift_analysis(train_ds, test_ds, train_df, test_df):
    """NIVEAU 2: Analyse de drift et distribution train/test"""
    print("\n" + "="*80)
    print("📈 NIVEAU 2: ANALYSE DRIFT ET DISTRIBUTION")
    print("="*80)
    
    results = {}
    
    if DEEPCHECKS_AVAILABLE:
        print("\n🔍 Exécution des checks Deepchecks:")
        print("   1. Train-Test Feature Drift (drift des features)")
        print("   2. Train-Test Label Drift (drift des labels)")
        print("   3. Whole Dataset Drift (drift global)")
        print("   4. Feature-Label Correlation (corrélations)")
        print("   5. Train-Test Samples Mix (fuites de données)")
        
        # Suite de validation train-test
        validation_suite = train_test_validation()
        
        print("\n⏳ Exécution de l'analyse de drift...")
        result = validation_suite.run(train_ds, test_ds)
        
        # Sauvegarder le rapport
        drift_report_path = REPORTS_DIR / 'crime_drift_analysis_report.html'
        result.save_as_html(str(drift_report_path))
        
        print(f"✅ Rapport de drift sauvegardé: {drift_report_path.name}")
        
        results['deepchecks_result'] = result
    
    # Analyses manuelles de drift spécifiques au crime
    print("\n📊 Analyses Drift Manuelles Spécifiques Crime:")
    
    # 1. Distribution temporelle (heures)
    if 'Hour' in train_df.columns:
        from scipy.stats import ks_2samp
        
        ks_stat, p_value = ks_2samp(train_df['Hour'], test_df['Hour'])
        print(f"   Distribution heures - KS test: p={p_value:.4f} {'✅ OK' if p_value > 0.05 else '⚠️ DRIFT'}")
        results['hour_drift_p'] = p_value
    
    # 2. Distribution géographique (coordonnées)
    if 'LAT' in train_df.columns and 'LON' in train_df.columns:
        # Test sur latitude
        lat_ks_stat, lat_p = ks_2samp(train_df['LAT'].dropna(), test_df['LAT'].dropna())
        # Test sur longitude
        lon_ks_stat, lon_p = ks_2samp(train_df['LON'].dropna(), test_df['LON'].dropna())
        
        print(f"   Distribution LAT - KS test: p={lat_p:.4f} {'✅ OK' if lat_p > 0.05 else '⚠️ DRIFT'}")
        print(f"   Distribution LON - KS test: p={lon_p:.4f} {'✅ OK' if lon_p > 0.05 else '⚠️ DRIFT'}")
        
        results['geo_drift'] = {'lat_p': lat_p, 'lon_p': lon_p}
    
    # 3. Distribution des labels (crimes)
    if hasattr(train_df, '__label__') or '__label__' in train_df.columns:
        from scipy.stats import chi2_contingency
        
        train_labels = train_df['__label__'] if '__label__' in train_df.columns else train_df.iloc[:, -1]
        test_labels = test_df['__label__'] if '__label__' in test_df.columns else test_df.iloc[:, -1]
        
        train_counts = train_labels.value_counts().sort_index()
        test_counts = test_labels.value_counts().sort_index()
        
        # Assurer même index
        all_labels = train_counts.index.union(test_counts.index)
        train_counts = train_counts.reindex(all_labels, fill_value=0)
        test_counts = test_counts.reindex(all_labels, fill_value=0)
        
        contingency_table = pd.DataFrame({
            'train': train_counts,
            'test': test_counts
        }).T
        
        chi2_stat, label_p, dof, expected = chi2_contingency(contingency_table)
        print(f"   Distribution labels - Chi2 test: p={label_p:.4f} {'✅ OK' if label_p > 0.05 else '⚠️ DRIFT'}")
        
        results['label_drift_p'] = label_p
    
    # 4. Statistiques descriptives
    print(f"\n📊 Comparaison statistiques Train vs Test:")
    for feature in features[:5]:  # Top 5 features
        if feature in train_df.columns:
            train_mean = train_df[feature].mean()
            test_mean = test_df[feature].mean()
            diff_pct = abs(train_mean - test_mean) / train_mean * 100 if train_mean != 0 else 0
            
            status = "✅ OK" if diff_pct < 5 else "⚠️ DIFF" if diff_pct < 15 else "🔴 DRIFT"
            print(f"   {feature}: Train={train_mean:.3f} Test={test_mean:.3f} Diff={diff_pct:.1f}% {status}")
    
    return results

In [ ]:
# Exécuter l'analyse de drift
drift_results = run_drift_analysis(train_dataset, test_dataset, 
                                  train_dataset if DEEPCHECKS_AVAILABLE else X_train, 
                                  test_dataset if DEEPCHECKS_AVAILABLE else X_test)

In [ ]:
# Afficher le widget interactif Deepchecks (si disponible)
if 'deepchecks_result' in drift_results:
    print("📊 Rapport interactif Deepchecks:")
    drift_results['deepchecks_result']

---
## 🎯 Analyse de Stabilité Temporelle (Spécifique Crime)

In [ ]:
def temporal_stability_analysis(df):
    """Analyse spécifique à la stabilité temporelle des crimes"""
    print("\n🕒 ANALYSE DE STABILITÉ TEMPORELLE")
    print("-" * 80)
    
    # Analyser les patterns temporels
    if 'DATE OCC' in df.columns:
        df_temp = df.copy()
        df_temp['DATE OCC'] = pd.to_datetime(df_temp['DATE OCC'], errors='coerce')
        df_temp = df_temp.dropna(subset=['DATE OCC'])
        
        # Grouper par mois
        monthly_counts = df_temp.groupby(df_temp['DATE OCC'].dt.to_period('M')).size()
        
        # Coefficient de variation
        cv = monthly_counts.std() / monthly_counts.mean()
        print(f"   Coefficient variation mensuel: {cv:.3f} {'✅ Stable' if cv < 0.2 else '⚠️ Variable'}")
        
        # Distribution par jour de la semaine
        if 'Day_of_week' in df.columns:
            day_dist = df['Day_of_week'].value_counts().sort_index()
            day_cv = day_dist.std() / day_dist.mean()
            print(f"   Variation jours semaine: {day_cv:.3f} {'✅ Équilibré' if day_cv < 0.3 else '⚠️ Déséquilibré'}")
        
        # Distribution par heure
        if 'Hour' in df.columns:
            hour_dist = df['Hour'].value_counts().sort_index()
            # Les heures de nuit devraient avoir moins de crimes
            night_crimes = hour_dist.loc[hour_dist.index.isin([0, 1, 2, 3, 4, 5])].sum()
            total_crimes = hour_dist.sum()
            night_pct = night_crimes / total_crimes * 100
            
            print(f"   Crimes nocturnes (0h-5h): {night_pct:.1f}% {'✅ Normal' if 5 <= night_pct <= 20 else '⚠️ Atypique'}")
        
        return {
            'monthly_cv': cv,
            'night_crimes_pct': night_pct if 'night_pct' in locals() else None
        }
    else:
        print("   ⚠️ Colonne DATE OCC manquante - analyse temporelle limitée")
        return {}

In [ ]:
# Exécuter l'analyse temporelle
temporal_results = temporal_stability_analysis(df_crime)

---
## 📊 Résumé et Recommandations

In [ ]:
print("\n" + "="*80)
print("✅ NIVEAU 2 : DRIFT ET DISTRIBUTION - TERMINÉ")
print("="*80)

print(f"\n📊 Résumé Drift Analysis:")
print(f"   Train samples: {X_train.shape[0]}")
print(f"   Test samples: {X_test.shape[0]}")
print(f"   Features analysées: {len(features)}")

# Résumé des tests statistiques
drift_detected = False

if 'hour_drift_p' in drift_results:
    hour_ok = drift_results['hour_drift_p'] > 0.05
    print(f"   Drift heures: {'✅ OK' if hour_ok else '⚠️ DÉTECTÉ'}")
    if not hour_ok:
        drift_detected = True

if 'geo_drift' in drift_results:
    geo_ok = (drift_results['geo_drift']['lat_p'] > 0.05 and 
              drift_results['geo_drift']['lon_p'] > 0.05)
    print(f"   Drift géographique: {'✅ OK' if geo_ok else '⚠️ DÉTECTÉ'}")
    if not geo_ok:
        drift_detected = True

if 'label_drift_p' in drift_results:
    label_ok = drift_results['label_drift_p'] > 0.05
    print(f"   Drift labels: {'✅ OK' if label_ok else '⚠️ DÉTECTÉ'}")
    if not label_ok:
        drift_detected = True

# Évaluation globale
print(f"\n🎯 Évaluation Globale:")
if drift_detected:
    print(f"   Status: ⚠️ DRIFT DÉTECTÉ - Vérifications nécessaires")
    print(f"   Action: Analyser les causes du drift et ajuster le split si nécessaire")
else:
    print(f"   Status: ✅ DISTRIBUTION STABLE - Données prêtes pour entraînement")
    print(f"   Action: Procéder au NIVEAU 3 (Performance du modèle)")

print(f"\n💡 Prochaines étapes:")
if not drift_detected:
    print(f"   1. ✅ Split train/test validé")
    print(f"   2. 🚀 Entraîner les modèles")
    print(f"   3. 🔄 Exécuter: deepchecks_performance.ipynb")
else:
    print(f"   1. 🔧 Investiguer les causes du drift détecté")
    print(f"   2. ⚖️ Considérer un re-split stratifié différent")
    print(f"   3. ♻️ Re-exécuter ce notebook après ajustements")

print(f"\n📋 Rapports générés:")
for report_file in REPORTS_DIR.glob('*drift*.html'):
    print(f"   📄 {report_file.name}")

print("\n" + "="*80)